#Vertex Agents SDK for quering MongoDB Atlas

This colab demonstrates all how to use Vertex AI extension to query MongoDB Atlas

##Authenticate

In [2]:
from google.colab import auth
auth.authenticate_user()

##Download most recent copy of Vertex Agents SDK

In [3]:
!gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl .

Copying gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl...
/ [1 files][  5.2 MiB/  5.2 MiB]                                                
Operation completed over 1 objects/5.2 MiB.                                      


In [4]:
# !gsutil cp gs://vertex_agents_private_releases/vertex_agents/google_cloud_aiplatform-1.55.dev20240609+vertex.agents-py2.py3-none-any.whl .

!pip install --upgrade --force-reinstall -q google_cloud_aiplatform-1.60.dev20240710+vertex.agents-py2.py3-none-any.whl --no-warn-conflicts
!pip install numpy==1.26.4

# Restart the kernel runtime to load the private preview SDK
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 3.1 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp310-cp310-manylin

{'status': 'ok', 'restart': True}

##Imports and Initialization

Make sure to initialize aiplatform with your projectID, location, and api endpoint. You need to initialize aiplatform before performing any of the other imports

In [1]:
# TODO: Replace with your project ID.
PROJECT_ID = "gcp-pov" # @param {type:"string"}
LOCATION = 'us-central1'
API_ENDPOINT = 'us-central1-aiplatform.googleapis.com'

from google.cloud import aiplatform
from google.cloud.aiplatform.private_preview.vertex_agents.app import App, Session
from google.cloud.aiplatform.private_preview.vertex_agents.agent import Agent
from vertexai.preview.extensions import Extension

aiplatform.init(project=PROJECT_ID, location=LOCATION, api_endpoint=API_ENDPOINT)

#Apps

##Create App

There are two options for creating an app.


1.   use the App.create function, which will create a new app for you
2.   initialize a Vertex Agents App object with the resource name of a previously created App



In [2]:
app = App.create(display_name='Vertex Agents App for MongoDB v1.6', description='made via the SDK')

# To load an existing app
# app = App("projects/<project_id>/locations<location_id>/apps/<app_id>")

##List Apps

You can list all the apps in a project, which will return a list of operational Apps.

Alternatively, you can get a user-friendly mapping of App display names and resource names.

In [3]:
all_apps = App.list_apps()
all_apps[0]

app_name: projects/787220387490/locations/us-central1/apps/2512445642119315456
display_name: Vertex Agents App for MongoDB v1.6
description: made via the SDK
agents: {}

# Tools

###Extension Tools

You can integrate both 1p and 3p Extension tools into your agent. To get a reference to a 1p extension, you can use the `Extensions.from_hub(...)` method

####3p Extensions

To create a 3p extension, you will need to specify the display name, description, and manifest of your extension.

In [13]:
## This is just a sample values please replace accordingly to your project
import os

# Setting up the GCP project
os.environ['PROJECT_ID'] = 'gcp-pov'  # GCP Project ID
os.environ['REGION'] =  "us-central1" # Project Region
## GCS Bucket location
os.environ['STAGING_BUCKET'] =  "gs://vertexai_extensions"
## Extension Config
os.environ['EXTENSION_DISPLAY_HOME'] =  "MongoDb Vertex API Interpreter"
os.environ['EXTENSION_DESCRIPTION'] =  "This extension makes api call to mongodb cluster FreeTier database sample_papers and collection embedding_papers to do all crud operations "

## OPEN API SPec config
os.environ['MANIFEST_NAME'] =  "mdb_crud_interpreter_for_agent_sdk"
os.environ['MANIFEST_DESCRIPTION'] =  """
This extension makes api call to mongodb cluster FreeTier, database sample_papers and collection embedding_papers to do all crud operations and This extension makes api call to mongodb cluster FreeTier, database sample_restaurants  and collection restaurants to do all crud operations based on the user query related to papers or resturants

"""
os.environ['OPENAPI_GCS_URI'] =  "gs://vertexai_extensions/mdb_extension_updated.yaml"

## API KEY secret location
# os.environ['API_SECRET_LOCATION'] = "projects/787220387490/secrets/mdbapi/versions/1"
os.environ['API_SECRET_LOCATION'] = "projects/787220387490/secrets/mdbapi/versions/1"


##LLM config
os.environ['LLM_MODEL'] = "gemini-1.5-pro"

from google.cloud import aiplatform
from vertexai.preview import extensions

PROJECT_ID = os.environ['PROJECT_ID']
REGION = os.environ['REGION']
STAGING_BUCKET = os.environ['STAGING_BUCKET']

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET,
)

In [14]:
mdb_crud = extensions.Extension.create(
         display_name = os.environ['EXTENSION_DISPLAY_HOME'],
         # Optional.
         description = os.environ['EXTENSION_DESCRIPTION'],  # Optional.
         manifest = {
             "name": os.environ['MANIFEST_NAME'],
             "description": os.environ['MANIFEST_DESCRIPTION'],
             "api_spec": {
                 "open_api_gcs_uri": (
                     os.environ['OPENAPI_GCS_URI']
                 ),
             },
             "auth_config": {
                 # GOOGLE_SERVICE_ACCOUNT_AUTH is only for 1P supported extensions.
                 "apiKeyConfig":{
             "name":"api-key",
             "apiKeySecret":os.environ['API_SECRET_LOCATION'],
             "httpElementLocation": "HTTP_IN_HEADER"
             },
             "authType":"API_KEY_AUTH"
             },
         },
     )
mdb_crud

INFO:vertexai.extensions._extensions:Creating Extension
INFO:vertexai.extensions._extensions:Create Extension backing LRO: projects/787220387490/locations/us-central1/extensions/5215256329425256448/operations/4302319902053629952
INFO:vertexai.extensions._extensions:Extension created. Resource name: projects/787220387490/locations/us-central1/extensions/5215256329425256448
INFO:vertexai.extensions._extensions:To use this Extension in another session:
INFO:vertexai.extensions._extensions:extension = vertexai.preview.extensions.Extension('projects/787220387490/locations/us-central1/extensions/5215256329425256448')


resource name: projects/787220387490/locations/us-central1/extensions/5215256329425256448

# Operations on an app

## Agents

Agents exist under an App. Agents can perform small, specific tasks utilizing Vertex Extensions and function calling. Agents require a display name, and a set of instructions.

When creating an agent, you must specify the following:


1.   Display Name
2.   Instructions - the detailed instructions the Agent should follow. This is where you instruct your agent when to call your functions / extensions and with what parameters.

You also have the option to specify the extensions and functions you will be using. Please note that if you reference an extension or function in the instructions, you ***must*** include an extension/function with the same display name in the extensions/functions lists.

##Create Agent
You can create an agent for your app by using the add_agent method. When creating an agent, you must specify all the required fields discussed above.



##Create a single Agent App

If you want to create an App and Agent in a single turn (i.e. avoiding using create_app -> add_agent) you can use `App.create_single_agent_app`. You must specify the agent display name, and then optionally specify tools for the agent, and the App's display name and description. This function returns an operable App.

In [15]:
DISPLAY_NAME = "MongoDB Agent"
INSTRUCTIONS = """
If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
The extension makes api call to mongodb cluster named 'FreeTier' database sample_papers and collection embedding_papers to do all crud operations only return the documents from the same.
Only respond based on the response of the tool. Do not create your own answers.
"""

app = App.create_single_agent_app(
    agent_display_name=DISPLAY_NAME,
    instructions=INSTRUCTIONS,
    extensions={
        'mongodb crud': mdb_crud}
    )

In [ ]:
# DISPLAY_NAME = "MDB Agent"
# INSTRUCTIONS = """
# If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
# The extension makes api call to mongodb cluster FreeTier database sample_papers and collection embedding_papers to do all crud operations only return the documents from the same.
# Only respond based on the response of the tool. Do not create your own answers.
# """

# updated_dinner_agent = created_agent.update(
#     new_display_name=DISPLAY_NAME,
#     new_instructions=INSTRUCTIONS,
#     new_extensions={
#         'mongodb': mdb_crud
#     }
# )

##List Agents

You can list all the agents exiting under an App. This will return operatable Agent objects.

In [16]:
app.list_agents()

[
 agent_name: projects/787220387490/locations/us-central1/apps/5971210155939856384/agents/1523778530318483456
 display_name: MongoDB Agent
 model: projects/787220387490/locations/us-central1/publishers/google/models/gemini-1.5-pro-001
 instructions: 
 If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
 The extension makes api call to mongodb cluster named 'FreeTier' database sample_papers and collection embedding_papers to do all crud operations only return the documents from the same.
 Only respond based on the response of the tool. Do not create your own answers.
 
 tools: { 
   extension_tool:{
 	extension: "projects/787220387490/locations/us-central1/extensions/5215256329425256448"
 	display_name: "mongodb crud"
   } }]

##Get Agent

You can also get a specific Agent using either the resource name or the display name

In [17]:
agent = app.get_agent('MongoDB Agent')

In [18]:
agent


agent_name: projects/787220387490/locations/us-central1/apps/5971210155939856384/agents/1523778530318483456
display_name: MongoDB Agent
model: projects/787220387490/locations/us-central1/publishers/google/models/gemini-1.5-pro-001
instructions: 
If user ask about anything from papers published use the extension MongoDb Vertex API Interpreter to make the call,
The extension makes api call to mongodb cluster named 'FreeTier' database sample_papers and collection embedding_papers to do all crud operations only return the documents from the same.
Only respond based on the response of the tool. Do not create your own answers.

tools: { 
  extension_tool:{
	extension: "projects/787220387490/locations/us-central1/extensions/5215256329425256448"
	display_name: "mongodb crud"
	extension_operations {
	  operation_id: "insertone_mdb"
	  function_declaration {
	    name: "insertone_mdb"
	    description: "\nThis extension makes api call to mongodb cluster FreeTier, database sample_papers and coll

Alternatively, you can get an Agent using the Agent class

Note that the only way to create a new Agent is to create it under an App, using `app.add_agent()`. The Agents constructor can only be used to get a reference to an Agent that has already been created.

In [ ]:
# agent2 = Agent(agent_id='projects/<project id>/locations/<location id>/<app id>/agents/<agent id>0')

## Delete Agent
Using either the display name or the fully qualified resource name, you can delete a specific agent under the App.

In [ ]:
# app.delete_agent('<Agent Name>')

#Session

NOTE: you can remove `.content` from the end of each run call to get more detailed information about each session action

In [19]:
session = app.start_session()

## Multi-turn queries without tool calls


In [22]:
session.run("Find the title by submitter 'Pavel Nadolsky'").content

role: "model"
parts {
  text: "```json\n{\"document\": {\"_id\": \"66706b69e3c7287461f83d8e\", \"abstract\": \"  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\\nmore detailed tests with CDF and DO data. Predictions are shown for\\ndistributions of diphoton pairs produced at the energy of the Large Hadron\\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\\nboson are contrasted with those produced from QCD proces

In [23]:
session.run("cuisine for resturant Wilken'S Fine Food").content

role: "model"
parts {
  text: "```json\n{\"document\": {\"_id\": \"5eb3d668b31de5d588f4292b\", \"address\": {\"building\": \"7114\", \"coord\": [-73.9068506, 40.6199034], \"street\": \"Avenue U\", \"zipcode\": \"11234\"}, \"borough\": \"Brooklyn\", \"cuisine\": \"Delicatessen\", \"grades\": [{\"date\": \"2014-05-29T00:00:00Z\", \"grade\": \"A\", \"score\": 10}, {\"date\": \"2014-01-14T00:00:00Z\", \"grade\": \"A\", \"score\": 10}, {\"date\": \"2013-08-03T00:00:00Z\", \"grade\": \"A\", \"score\": 8}, {\"date\": \"2012-07-18T00:00:00Z\", \"grade\": \"A\", \"score\": 10}, {\"date\": \"2012-03-09T00:00:00Z\", \"grade\": \"A\", \"score\": 13}, {\"date\": \"2011-10-14T00:00:00Z\", \"grade\": \"A\", \"score\": 9}], \"name\": \"Wilken\'S Fine Food\", \"restaurant_id\": \"40356483\"}}\n```\n"
}

In [ ]:
session.get_history()

name: "projects/787220387490/locations/us-central1/apps/7451205583484485632/sessions/467402942723391488"
actions {
  message {
    role: "user"
    parts {
      text: "can you repeat my word"
    }
  }
  create_time {
    seconds: 1719299849
    nanos: 818392000
  }
  turn: 1
}
actions {
  message {
    role: "model"
    parts {
      text: "I am sorry, I cannot fulfill this request. The available tools lack the desired functionality. \n"
    }
  }
  create_time {
    seconds: 1719299849
    nanos: 818392000
  }
  turn: 1
}
actions {
  message {
    role: "user"
    parts {
      text: "Find the title by submitter \'Pavel Nadolsky\'"
    }
  }
  create_time {
    seconds: 1719299889
    nanos: 835159000
  }
  turn: 2
}
actions {
  tool_use {
    extension_invocation {
      extension_name: "projects/787220387490/locations/us-central1/extensions/1815196940434931712"
      operation_id: "findmany_mdb"
    }
    action: "findmany_mdb"
    input_parameters {
      fields {
        key: "f

In [ ]:
session.run("hi")

session: "projects/787220387490/locations/us-central1/apps/425027214833090560/sessions/879763783604502528"
content {
  role: "model"
  parts {
    text: "Hello! 👋 How can I help you today? \n"
  }
}
actions {
  message {
    role: "user"
    parts {
      text: "hi"
    }
  }
  create_time {
    seconds: 1723829446
    nanos: 363725000
  }
  turn: 2
}
actions {
  message {
    role: "model"
    parts {
      text: "Hello! 👋 How can I help you today? \n"
    }
  }
  create_time {
    seconds: 1723829446
    nanos: 363725000
  }
  turn: 2
}

## User query that invokes client side function


In [24]:
session.run("Find the title by submitter 'Pavel Nadolsky'").content

role: "model"
parts {
  text: "```json\n{\"document\": {\"_id\": \"66706b69e3c7287461f83d8e\", \"abstract\": \"  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\\nmore detailed tests with CDF and DO data. Predictions are shown for\\ndistributions of diphoton pairs produced at the energy of the Large Hadron\\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\\nboson are contrasted with those produced from QCD proces

In [ ]:
# app.delete_session('<Session Name>')